In [ ]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 875.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# ================== 1️⃣ Load Dataset ==================
df = pd.read_csv("/content/drive/MyDrive/SMOTE4.csv")  # Replace with actual path

In [ ]:
# Define Features & Target
feature_cols = ["durata", "DistTOT", "HSR", "ACC", "DEC", "RPE", "sRPE"]
target_col = "phase"

# Convert Target to Categorical Labels
label_encoder = LabelEncoder()
df[target_col] = label_encoder.fit_transform(df[target_col])  # Encode phase (1-4)

# Train-Test Split (70-30 for better generalization)
X_train, X_test, y_train, y_test = train_test_split(
    df[feature_cols], df[target_col],
    test_size=0.3,
    random_state=42,
    stratify=df[target_col]  # Ensures balanced class distribution
)

In [ ]:
# ================== 2️⃣ Feature Scaling ==================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# ================== 3️⃣ Train TabNet Model ==================
tabnet = TabNetClassifier(
    lambda_sparse=1e-3,
    momentum=0.98,
    verbose=10,
    seed=42
)

# Train TabNet
tabnet.fit(
    X_train_scaled, y_train.values,
    eval_set=[(X_test_scaled, y_test.values)],
    eval_metric=['accuracy'],
    max_epochs=30,  # ✅ Correct placement inside fit()
    patience=10
)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.61482 | val_0_accuracy: 0.31393 |  0:00:00s
epoch 10 | loss: 0.8418  | val_0_accuracy: 0.7082  |  0:00:02s
epoch 20 | loss: 0.56257 | val_0_accuracy: 0.7918  |  0:00:04s
Stop training because you reached max_epochs = 30 with best_epoch = 24 and best_val_0_accuracy = 0.86557


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
# ================== 4️⃣ Extract TabNet Feature Embeddings ==================
embeddings_train = tabnet.predict_proba(X_train_scaled)
embeddings_test = tabnet.predict_proba(X_test_scaled)

In [ ]:
# ================== 5️⃣ Train Random Forest on TabNet Embeddings ==================
rf = RandomForestClassifier(
    n_estimators=50,  # Reduced to avoid overfitting
    max_depth=10,
    min_samples_split=10,
    max_features="sqrt",
    random_state=42
)

rf.fit(embeddings_train, y_train)


RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=50,
                       random_state=42)

In [ ]:
# ================== 6️⃣ Evaluate Hybrid Model ==================
train_preds = rf.predict(embeddings_train)
test_preds = rf.predict(embeddings_test)

train_acc = accuracy_score(y_train, train_preds)
test_acc = accuracy_score(y_test, test_preds)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


Train Accuracy: 0.9582
Test Accuracy: 0.9000


In [ ]:
# Classification Report
print("\nClassification Report:\n", classification_report(y_test, test_preds))

# ================== 7️⃣ Cross-Validation for Stability ==================
cv_scores = cross_val_score(rf, embeddings_train, y_train, cv=5)
print(f"\nCross-Validation Accuracy: {np.mean(cv_scores):.4f}")


Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.90      0.91       305
           1       0.87      0.93      0.90       305
           2       0.89      0.88      0.88       305
           3       0.93      0.90      0.91       305

    accuracy                           0.90      1220
   macro avg       0.90      0.90      0.90      1220
weighted avg       0.90      0.90      0.90      1220


Cross-Validation Accuracy: 0.8938
